<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/uHooProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@Created on 2018-12-14
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

#FIRST STEP: PREPARING THE DATA
print("Preparing the data...")

#SECOND STEP: TRAINING THE MODEL WITH LSTM
print("Training the model...")

model = Sequential()
model.add(LSTM(128, activation='softmax', return_sequences = True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(256, activation='softmax', dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history = model.fit(input_train, y_train,epochs=10,batch_size=128, validation_split=0.2)

model.save('lstm_model.h5')

#THIRD STEP: PLOT RESULTS
print("Plot results...")

In [0]:
'''
@Description: Generator method for timestep forecasting.
@Source code: "Deep Learning with Python", Francois Chollet
'''
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=64, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  
  while 1:
    if shuffle:
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
        if i + batch_size >= max_index:
          i = min_index + lookback
        rows = np.arange(i, min(i + batch_size, max_index))
        i += len(rows)
        
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))
    
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][1]
    yield samples, targets